In [2]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import pandas as pd
import numpy as np
import face_recognition
import sqlite3

# 1. Function for Fetching all the Image Embeddings from the SQLite Database

In [3]:
def get_embeddings():
    
    #connecting to sqlite
    conn = sqlite3.connect("facial_recognition.db")
    
    #querying to fetch all image embeddings
    cur = conn.cursor()
    cur.execute("SELECT * FROM embeds")
    
    #fetching all embeddings
    rows = cur.fetchall()
    
    #creating a list of all image names
    image_names = [row[0] for row in rows]
    
    #creating a list of all image embeddings
    image_embeds = [row[1:] for row in rows]
    #converting list to numpy array
    image_embeds = np.array(image_embeds)
    
    return image_names, image_embeds

# 2. Function for displaying image

In [4]:
def display_image(img, row, col):
    global tk_img
    
    img_resized = img.resize((150, 150)) # new width & height
    tk_img = ImageTk.PhotoImage(img_resized)
    
    #displaying the input image using the label widget
    img_label = tk.Label(input_img_frame, image=tk_img, padx=10, pady=10)
    img_label.grid(row=row, column=col, pady=2)

# 3. Function for uploading .jpg images in the tkinter application

In [5]:
def upload_file():
    global matched_imgs
    # Opening the window to select the input image
    f_types = [('Jpg Files', '*.jpg')]
    filename = filedialog.askopenfilename(filetypes=f_types)
    
    #reading the input image
    input_img = Image.open(filename)
    
    inp_img_label = tk.Label(input_img_frame, text="Input Image", font=('Times New Roman',18,'bold', 'italic', 'underline'), anchor="center", bg="black", fg="orange")
    inp_img_label.grid(row=0, column=0)
    
    #displaybing the input image
    display_image(input_img, 1, 0)
    
    img_name = tk.Label(input_img_frame, text=f"{filename.split('/')[-1].split('.')[0]}", font=("Times New Roman", 14), fg="#39FF14", bg="black")
    img_name.grid(row=2, column=0, pady=5)
    
    results = match(input_img)
    sorted_results_idx = np.argsort(results)
    
    # coordinates of matched images on the gui
    x, y = 0, 0
    
    #Displaying the top 6 matched images from our database
    matched_imgs = []
    for i, idx in enumerate(sorted_results_idx[:10]):
            
        match_image_name = image_names[idx]
        matched_img = Image.open(f"Train/{match_image_name}.jpg")
        matched_img = matched_img.resize((130,130)) # new width & height
        img2 = ImageTk.PhotoImage(matched_img)
        matched_imgs.append(img2)

        matched_img_label = tk.Label(matches_frame, image=matched_imgs[-1], padx=10, pady=10) # using Button 
        matched_img_label.grid(row=x, column=y, padx=15, pady=5)

        match_img_name = tk.Label(matches_frame, text=f"{match_image_name} - {results[idx]: .2f}", font=("Times New Roman", 14), fg="#39FF14", bg="black")
        match_img_name.grid(row=x+1, column=y, pady=5)

        if y >= 4:
            x += 2
            y = 0
        else:
            y += 1

# 4. Backend code for the Reset button

In [6]:
def reset():
    # looping through the childern(widgets) of the last two frame and destroying them if they are labels
    for frame in root.winfo_children()[-2:]:
        for widget in frame.winfo_children():
            if isinstance(widget, tk.Label):
                widget.destroy()

# 5. Function to match the input image embedding to the embeddings of all other images

In [7]:
 def match(input_img):
    #converting input image to NumPy array
    input_img = np.array(input_img, dtype=np.uint8)
    #generating the input image embedding
    encoding = face_recognition.face_encodings(input_img)[0]
    
    #calculating the distance between input image and all the database images
    results = face_recognition.api.face_distance(image_embeds, encoding)
    
    return results

# 6. Main tkinter GUI code

In [8]:
root = tk.Tk()
root.configure(bg='black')
root.geometry("910x780")

image_names, image_embeds = get_embeddings()

label = tk.Label(root, text="Facial Recognition", font=("Times New Roman bold", 34), height=2, anchor="s", bg="white")
label.pack(side="top", fill='both')

canvas = tk.Canvas(root, bg='orange', height=3.5) ##FCC201
canvas.pack(side="top", fill='both')

body = tk.Frame(root, bg="black")
body.pack(side="top")

button = tk.Button(body, text="Upload", command=lambda: upload_file(), borderwidth=5, relief='ridge', padx=6, pady=5, font=("Times New Roman Bold", 13), fg='black', bg='white')
button.grid(row=0, column=0, padx=7, pady=10)

reset_button = tk.Button(body, text="Reset", command=lambda: reset(), borderwidth=5, relief='ridge', padx=6, pady=5, font=("Times New Roman Bold", 13), fg='black', bg='white')
reset_button.grid(row=0, column=1)

input_img_frame = tk.Frame(root, bg="black")
input_img_frame.pack(side='top')

matches_frame = tk.Frame(root, bg="black")
matches_frame.pack(side='top')

root.mainloop()